In [1]:
import datetime

import pandas as pd

from WorkflowMSC import WorkflowMSC

In [2]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"

In [3]:
start_time = datetime.datetime(2000, 1, 1)
end_time = datetime.datetime(2001, 1, 10)

msc = WorkflowMSC(connection_string, start_time, end_time)

In [4]:
precip = msc.annual_rain()

Year interval: 1.0


In [5]:
pd.read_sql(precip.statement, msc.engine)

,sum,max_date,lat,lon,station_id
0,648.8,2001-01-10,48.5733,-123.3920,16
1,1102.9,2001-01-10,48.9350,-123.7420,18
2,552.1,2001-01-10,48.4178,-123.4810,22
3,787.3,2001-01-10,48.7111,-123.5570,27
4,1805.2,2001-01-10,48.8244,-124.1330,29
5,1687.3,2001-01-10,48.8294,-124.0520,32
6,1033.2,2001-01-10,48.7347,-123.7280,39
7,630.8,2001-01-10,48.4320,-123.4390,45
8,878.8,2001-01-10,48.8717,-123.4980,48
9,812.1,2001-01-10,48.9850,-123.5730,50
